In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
I_GPU = 0

In [42]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.s3dis import S3DISFusedDataset
# from torch_points3d.datasets.segmentation.multimodal.s3dis import S3DISFusedDataset
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

# Load config

In [43]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
DATA_ROOT = '/raid/dataset/pointcloud/data/s3dis'  # ENGIE

# overrides = [
# #     'model_name=XYZ-RGB-pyramid-0_out-128',
# #     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-08-26/13-57-08'}",
# #     'model_name=ResUNet34-L4-L0-128',
# #     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-09-24/16-46-01'}",
    
#     #'model_name=ResUNet34-L4-L0-96_Emod',
#     'model_name=Res16UNet34-L4-early',
#     #f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-09-27/08-55-56'}",
#     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-08/21-00-14'}",

    
# #    'voting_runs=10',
#     'voting_runs=5',
# #     'voting_runs=3',
# #     'voting_runs=1',
#    'tracker_options.full_res=False',
# #     'tracker_options.full_res=False',
#     'precompute_multi_scale=False',
#     'num_workers=4',
#     'batch_size=4',
#     f'cuda={I_GPU}',
#     'weight_name=miou',
# ]

# XYZRGB*
overrides = [
    'model_name=Res16UNet34',
    f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-11/22-27-43'}",
    
    'voting_runs=1',
    'tracker_options.full_res=True',
    'precompute_multi_scale=False',
    'num_workers=4',
    'batch_size=8',
    f'cuda={I_GPU}',
    'weight_name=last',
]

# XYZ+RGB
# overrides = [
#     'model_name=Res16UNet34-L4-early',
# #     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-09/21-34-55'}",  # Fold 2
# #     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-09/21-35-06'}",  # Fold 3
# #     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-20/18-26-24'}",  # Fold 5
#     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-17/18-13-17'}",
    
#     'voting_runs=1',
#     'tracker_options.full_res=True',
#     'precompute_multi_scale=False',
#     'num_workers=4',
#     'batch_size=4',
#     f'cuda={I_GPU}',
#     'weight_name=last',
# ]


cfg = hydra_read(overrides, config_name='eval')

OmegaConf.set_struct(cfg, False)

# print(OmegaConf.to_yaml(cfg))# Load S3DIS dataset

In [44]:
from torch_points3d.trainer import Trainer

trainer = Trainer(cfg)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# trainer.eval(stage_name="test")

In [64]:
# n = 5
# indices = [
#     np.random.choice(torch.where(trainer._dataset.test_dataset[0]._datas[0].y == i)[0].numpy(), n)
#     for i, c in enumerate(CLASSES[:-1])]
# originids = torch.cat([trainer._dataset.test_dataset[0]._datas[0].origin_id[v].view(1, -1) for v in indices], dim=0)
# originids

In [7]:
# trainer._dataset.test_dataset[0]._test_spheres  = [trainer._dataset.test_dataset[0]._test_spheres[i] for i in range(0, 1000, 10)]

In [46]:
# Update the val and test transforms to match train transforms for inference-time augmentation
trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image
# trainer._dataset.val_dataset.transform = trainer._dataset.train_dataset.transform
# trainer._dataset.val_dataset.transform_image = trainer._dataset.train_dataset.transform_image

In [32]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = 2097152 * 2

In [33]:
trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.8, 1.2]
trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.8, 1.2]
trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.8, 1.2]

In [22]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.8, 1.2]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.8, 1.2]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.8, 1.2]

In [20]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 * 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.9, 1.1]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.9, 1.1]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.9, 1.1]

In [6]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.01
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.95, 1.05]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.95, 1.05]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.95, 1.05]

In [42]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 4)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.001
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.99, 1.01]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.99, 1.01]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.99, 1.01]

In [46]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.2
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.1, 1.9]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.1, 1.9]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.1, 1.9]

In [79]:
# Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# trainer._dataset.test_dataset[0].transform_image.transforms[-1].mean = [0, 0, 0]
# trainer._dataset.test_dataset[0].transform_image.transforms[-1].std = [1, 1, 1]

In [14]:
# trainer._cfg.tracker_options.full_res = False

In [47]:
trainer._cfg.voting_runs = 1

In [35]:
# trainer._cfg.voting_runs = 5

In [48]:
# trainer.eval()
trainer.eval(stage_name="test")

  0%|          | 0/105 [00:06<?, ?it/s]

In [ ]:
# trainer._tracker._compute_full_miou()

# XYZRGB* 2cm Fold2

In [49]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 1.4974905428432277,
 'test_loss_cross_entropy': 1.4974905428432277,
 'test_acc': 73.20641906675,
 'test_macc': 60.77379592755784,
 'test_miou': 42.99125403762488,
 'test_miou_per_class': {0: '86.05',
  1: '44.70',
  2: '79.90',
  3: '21.93',
  4: '42.79',
  5: '39.67',
  6: '57.50',
  7: '29.46',
  8: '16.18',
  9: '46.10',
  10: '40.30',
  11: '16.31',
  12: '37.99'},
 'test_iou': {'ceiling': 0.8604966295843016,
  'floor': 0.446961359640673,
  'wall': 0.7990165029438703,
  'beam': 0.21932225673535483,
  'column': 0.42794358959917195,
  'window': 0.3967292758818588,
  'door': 0.5749776286212999,
  'chair': 0.29462998110820077,
  'table': 0.16179780839342003,
  'bookcase': 0.4609570001677765,
  'sofa': 0.40304567788851914,
  'board': 0.16307980661719323,
  'clutter': 0.37990550770959464},
 'test_full_vote_miou': 47.58133821240307,
 'test_full_vote_iou': {'ceiling': 87.09063195445772,
  'floor': 49.09351821003553,
  'wall': 80.45870711525764,
  'beam': 22.93279552099221

In [50]:
# torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/s3dis/results/sparseconv3d_fold2_2cm.pt')

# Res16UNet34-L4-max-early 5cm Fold5

In [37]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5296356549113994,
 'test_loss_cross_entropy': 0.5296356549113994,
 'test_acc': 88.76396806849746,
 'test_macc': 73.03362593076177,
 'test_miou': 65.81321433113524,
 'test_miou_per_class': {0: '89.89',
  1: '97.64',
  2: '84.00',
  3: '1.65',
  4: '21.77',
  5: '70.18',
  6: '68.44',
  7: '85.57',
  8: '77.64',
  9: '76.90',
  10: '44.98',
  11: '80.70',
  12: '56.21'},
 'test_iou': {'ceiling': 0.8989129174697266,
  'floor': 0.9764229314153903,
  'wall': 0.8400122081295054,
  'beam': 0.01654342807761926,
  'column': 0.21768434986384683,
  'window': 0.7017856545657677,
  'door': 0.6844256136163502,
  'chair': 0.8557323008298734,
  'table': 0.7763986270177725,
  'bookcase': 0.7689776124009458,
  'sofa': 0.4497578733747513,
  'board': 0.8069680731362285,
  'clutter': 0.5620962731498035},
 'test_full_vote_miou': 65.94583820145267,
 'test_full_vote_iou': {'ceiling': 86.69599083977323,
  'floor': 96.90220015496206,
  'wall': 83.87838824892917,
  'beam': 1.7698867645628897,

In [39]:
# torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/s3dis/results/Res16UNet34-L4-max-early_fold5_5cm.pt')

# Res16UNet34-mean-RGB-early 5cm Fold5

In [25]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.619402181063319,
 'test_loss_cross_entropy': 0.619402181063319,
 'test_acc': 84.87074826945471,
 'test_macc': 65.77387227845016,
 'test_miou': 57.92401359374689,
 'test_miou_per_class': {0: '90.36',
  1: '97.42',
  2: '76.77',
  3: '0.08',
  4: '19.32',
  5: '39.07',
  6: '38.37',
  7: '85.78',
  8: '75.49',
  9: '71.58',
  10: '58.10',
  11: '48.88',
  12: '51.79'},
 'test_iou': {'ceiling': 0.9036013830205673,
  'floor': 0.9741603126893063,
  'wall': 0.7676900948091477,
  'beam': 0.0008478400204202524,
  'column': 0.19318918566732252,
  'window': 0.39072123671683323,
  'door': 0.38373353850371705,
  'chair': 0.8578334020089592,
  'table': 0.7548623171764107,
  'bookcase': 0.7158264565038678,
  'sofa': 0.5809535794058871,
  'board': 0.48880119073432315,
  'clutter': 0.5179012299303348},
 'test_full_vote_miou': 60.52797579269067,
 'test_full_vote_iou': {'ceiling': 87.20800830754727,
  'floor': 96.74148872290198,
  'wall': 77.28105157924043,
  'beam': 1e-06,
  'column

In [26]:
# torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/s3dis/results/Res16UNet34-mean-RGB-early_fold5_5cm.pt')

# 2cm Fold 1

In [9]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5496826406774212,
 'test_loss_cross_entropy': 0.5496826406774212,
 'test_acc': 91.0511636892235,
 'test_macc': 89.09887117165812,
 'test_miou': 79.91465116296852,
 'test_miou_per_class': {0: '94.96',
  1: '96.31',
  2: '85.96',
  3: '75.81',
  4: '68.69',
  5: '82.88',
  6: '88.55',
  7: '85.20',
  8: '72.06',
  9: '62.37',
  10: '74.41',
  11: '79.51',
  12: '72.19'},
 'test_iou': {'ceiling': 0.9496460226647868,
  'floor': 0.9630784090629917,
  'wall': 0.8596480687972254,
  'beam': 0.7581099576764071,
  'column': 0.6868560083969494,
  'window': 0.828805076459369,
  'door': 0.8855129665748627,
  'chair': 0.8520359949198573,
  'table': 0.7205517281800041,
  'bookcase': 0.6236578785196787,
  'sofa': 0.7440637619130935,
  'board': 0.795054117575059,
  'clutter': 0.7218846604456224},
 'test_full_vote_miou': 80.29586908632251,
 'test_full_vote_iou': {'ceiling': 94.23636593196576,
  'floor': 96.12936532341794,
  'wall': 85.91647917204061,
  'beam': 76.14086848123122,
  'c

In [11]:
torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/s3dis/results/xyz+rgb_fold1_2cm.pt')

# Fold 5

In [10]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5561665276175646,
 'test_loss_cross_entropy': 0.5561665276175646,
 'test_acc': 89.076063455651,
 'test_macc': 73.15035293225608,
 'test_miou': 66.84508879336215,
 'test_miou_per_class': {0: '90.32',
  1: '97.77',
  2: '84.26',
  3: '0.26',
  4: '21.82',
  5: '67.33',
  6: '70.28',
  7: '85.89',
  8: '80.03',
  9: '76.84',
  10: '52.94',
  11: '82.83',
  12: '58.42'},
 'test_iou': {'ceiling': 0.9031783333013115,
  'floor': 0.9777280913923208,
  'wall': 0.8425893045174863,
  'beam': 0.0025623068240094265,
  'column': 0.21816525458024716,
  'window': 0.6733025323150481,
  'door': 0.7027529165238106,
  'chair': 0.8589462281921832,
  'table': 0.8002746895897181,
  'bookcase': 0.7684146003063445,
  'sofa': 0.5293793132969986,
  'board': 0.8283203828326371,
  'clutter': 0.5842475894649656},
 'test_full_vote_miou': 66.55803240958792,
 'test_full_vote_iou': {'ceiling': 86.79427603474079,
  'floor': 96.88171293863745,
  'wall': 83.79624428934102,
  'beam': 0.0229281749551866,

In [11]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5560733042783305,
 'test_loss_cross_entropy': 0.5560733042783305,
 'test_acc': 89.08134094975283,
 'test_macc': 73.1081531046332,
 'test_miou': 66.7727986052828,
 'test_miou_per_class': {0: '90.35',
  1: '97.80',
  2: '84.22',
  3: '0.26',
  4: '21.78',
  5: '67.25',
  6: '70.36',
  7: '85.80',
  8: '79.99',
  9: '76.97',
  10: '52.32',
  11: '82.45',
  12: '58.48'},
 'test_iou': {'ceiling': 0.9034638629033545,
  'floor': 0.9780060512143244,
  'wall': 0.8422461835685947,
  'beam': 0.0026302951524088845,
  'column': 0.2178221475963958,
  'window': 0.6725101860494516,
  'door': 0.7036040790602138,
  'chair': 0.8580426678079786,
  'table': 0.7999439329920773,
  'bookcase': 0.7696835137929061,
  'sofa': 0.5232234368696432,
  'board': 0.8244993840773719,
  'clutter': 0.584788077602045},
 'test_full_vote_miou': 66.50957590860219,
 'test_full_vote_iou': {'ceiling': 86.69959022189181,
  'floor': 96.8806209430931,
  'wall': 83.73557317167494,
  'beam': 0.03549435767163393,
 

In [12]:
# torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/s3dis/results/xyz+rgb_fold5_5cm.pt')

# Fold 2

In [13]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.6042165702722726,
 'test_loss_cross_entropy': 0.6042165702722726,
 'test_acc': 86.62172385247712,
 'test_macc': 73.87255233870636,
 'test_miou': 59.73326652513678,
 'test_miou_per_class': {0: '88.09',
  1: '91.38',
  2: '86.10',
  3: '28.36',
  4: '60.90',
  5: '40.38',
  6: '76.51',
  7: '65.11',
  8: '52.35',
  9: '49.75',
  10: '48.96',
  11: '34.12',
  12: '54.52'},
 'test_iou': {'ceiling': 0.8808912855533134,
  'floor': 0.913806440411782,
  'wall': 0.8609729752396305,
  'beam': 0.28360039258073455,
  'column': 0.6090013781381708,
  'window': 0.40383821259779207,
  'door': 0.7651401710675365,
  'chair': 0.6511212723351703,
  'table': 0.5234675114298487,
  'bookcase': 0.4974771470756293,
  'sofa': 0.489609988318565,
  'board': 0.3412365688287967,
  'clutter': 0.5451613046908107},
 'test_full_vote_miou': 61.72339478597231,
 'test_full_vote_iou': {'ceiling': 87.99840697594014,
  'floor': 92.79455935648578,
  'wall': 85.71214566942868,
  'beam': 30.61144350793974,
 

In [11]:
# torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/s3dis/results/xyz+rgb_fold2_5cm.pt')

# Fold 3

In [ ]:
# torch.save(trainer._tracker._full_confusion, '/raid/dataset/pointcloud/data/s3dis/results/xyz+rgb_fold3_5cm.pt')

In [28]:
# vote=1 - r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3218829301032392,
 'test_loss_cross_entropy': 0.3218829301032392,
 'test_acc': 92.17379274380939,
 'test_macc': 88.67533478124531,
 'test_miou': 80.24567261374068,
 'test_miou_per_class': {0: '95.26',
  1: '97.74',
  2: '86.78',
  3: '80.29',
  4: '57.76',
  5: '77.05',
  6: '86.04',
  7: '82.75',
  8: '73.40',
  9: '77.76',
  10: '70.95',
  11: '80.15',
  12: '77.25'},
 'test_iou': {'ceiling': 0.9525949237848576,
  'floor': 0.9774153720379403,
  'wall': 0.8678066059797175,
  'beam': 0.8028754539567391,
  'column': 0.577649015043894,
  'window': 0.7704882781812028,
  'door': 0.860355250217941,
  'chair': 0.8275335313497437,
  'table': 0.7340402471522532,
  'bookcase': 0.7775678444664276,
  'sofa': 0.7095196587351991,
  'board': 0.8015417033273533,
  'clutter': 0.7725495555530192},
 'test_full_vote_miou': 79.43889095365549,
 'test_full_vote_iou': {'ceiling': 93.05327299018775,
  'floor': 96.86439762827396,
  'wall': 86.21196824360833,
  'beam': 78.81340294597784,
  '

In [24]:
# vote=2 - r/2 - half credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.32217988677241005,
 'test_loss_cross_entropy': 0.32217988677241005,
 'test_acc': 92.16843603926614,
 'test_macc': 88.74374191653285,
 'test_miou': 80.27448058011092,
 'test_miou_per_class': {0: '95.24',
  1: '97.73',
  2: '86.82',
  3: '80.09',
  4: '58.32',
  5: '77.02',
  6: '85.86',
  7: '82.86',
  8: '73.55',
  9: '77.65',
  10: '71.22',
  11: '79.96',
  12: '77.24'},
 'test_iou': {'ceiling': 0.9523502435448606,
  'floor': 0.9773158940252332,
  'wall': 0.8682261373780679,
  'beam': 0.800886464234754,
  'column': 0.5832141972905062,
  'window': 0.7702413749277044,
  'door': 0.858560524222882,
  'chair': 0.8286383046147898,
  'table': 0.7355139716938428,
  'bookcase': 0.7764773286496915,
  'sofa': 0.7122210702614769,
  'board': 0.7996181929114184,
  'clutter': 0.7724187716591904},
 'test_full_vote_miou': 79.46430343185715,
 'test_full_vote_iou': {'ceiling': 93.01678965531283,
  'floor': 96.86740375032205,
  'wall': 86.27793428107624,
  'beam': 78.68906933418907,
 

In [17]:
# r/2 - x2 credit - aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.32423276727261763,
 'test_loss_cross_entropy': 0.32423276727261763,
 'test_acc': 92.13096120186151,
 'test_macc': 88.63534181220459,
 'test_miou': 80.12543931555223,
 'test_miou_per_class': {0: '95.24',
  1: '97.72',
  2: '86.79',
  3: '80.17',
  4: '57.90',
  5: '76.54',
  6: '85.50',
  7: '82.71',
  8: '73.44',
  9: '77.61',
  10: '70.80',
  11: '79.99',
  12: '77.22'},
 'test_iou': {'ceiling': 0.9523621916554682,
  'floor': 0.9772062599186939,
  'wall': 0.8678528749842629,
  'beam': 0.8017367809730164,
  'column': 0.5790070741610173,
  'window': 0.7653845774628152,
  'door': 0.8550234892834504,
  'chair': 0.8271040311928374,
  'table': 0.7344202614722822,
  'bookcase': 0.7761313025632556,
  'sofa': 0.7080480024129251,
  'board': 0.7998728571393379,
  'clutter': 0.772157407802427},
 'test_full_vote_miou': 79.34020273949261,
 'test_full_vote_iou': {'ceiling': 93.04855015032344,
  'floor': 96.83356527079596,
  'wall': 86.25682723150922,
  'beam': 79.3861384454172,
 

In [10]:
# r/2 - x2 credit - soft aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.32033917881384655,
 'test_loss_cross_entropy': 0.32033917881384655,
 'test_acc': 92.1756755619912,
 'test_macc': 88.69039884391421,
 'test_miou': 80.26560942653592,
 'test_miou_per_class': {0: '95.24',
  1: '97.73',
  2: '86.77',
  3: '80.31',
  4: '57.34',
  5: '77.24',
  6: '85.87',
  7: '82.89',
  8: '73.64',
  9: '77.75',
  10: '71.06',
  11: '80.17',
  12: '77.43'},
 'test_iou': {'ceiling': 0.9524006588095312,
  'floor': 0.977349392146094,
  'wall': 0.8677380579414667,
  'beam': 0.8031411158228586,
  'column': 0.5734415119772075,
  'window': 0.7723612000700912,
  'door': 0.8586781568676222,
  'chair': 0.8288727394371915,
  'table': 0.7363507685722425,
  'bookcase': 0.7774822927276696,
  'sofa': 0.7106345379486098,
  'board': 0.8017359103179438,
  'clutter': 0.774342882811141},
 'test_full_vote_miou': 79.41217429236951,
 'test_full_vote_iou': {'ceiling': 93.00448256064946,
  'floor': 96.84141562498583,
  'wall': 86.18880176550107,
  'beam': 78.96665590126676,
  

In [11]:
# r/3
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.32959187582931837,
 'test_loss_cross_entropy': 0.32959187582931837,
 'test_acc': 92.03387767904175,
 'test_macc': 88.45569833525056,
 'test_miou': 79.84800904072762,
 'test_miou_per_class': {0: '95.01',
  1: '97.63',
  2: '86.82',
  3: '80.89',
  4: '54.38',
  5: '75.06',
  6: '85.53',
  7: '83.08',
  8: '73.14',
  9: '77.64',
  10: '71.96',
  11: '79.88',
  12: '76.99'},
 'test_iou': {'ceiling': 0.9501456226119098,
  'floor': 0.9763337502222753,
  'wall': 0.8681684885171153,
  'beam': 0.8088795870868684,
  'column': 0.5438289220374718,
  'window': 0.7506065103582978,
  'door': 0.8552539942147027,
  'chair': 0.830848005581853,
  'table': 0.7314455667607253,
  'bookcase': 0.7764010108705107,
  'sofa': 0.7196382408102228,
  'board': 0.7987620987878549,
  'clutter': 0.7699293774347838},
 'test_full_vote_miou': 79.27449574309561,
 'test_full_vote_iou': {'ceiling': 93.01302600823149,
  'floor': 96.85067198094053,
  'wall': 86.24307576239741,
  'beam': 79.00348679945118,


In [10]:
# r/2
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3202797946886538,
 'test_loss_cross_entropy': 0.3202797946886538,
 'test_acc': 92.16376306359699,
 'test_macc': 88.7047623261769,
 'test_miou': 80.25090783194267,
 'test_miou_per_class': {0: '95.27',
  1: '97.73',
  2: '86.78',
  3: '80.15',
  4: '57.67',
  5: '76.74',
  6: '85.90',
  7: '82.88',
  8: '73.55',
  9: '77.76',
  10: '71.44',
  11: '80.14',
  12: '77.26'},
 'test_iou': {'ceiling': 0.9526917391413627,
  'floor': 0.9772913804612737,
  'wall': 0.8677805398725159,
  'beam': 0.8015015385392836,
  'column': 0.5766684493327823,
  'window': 0.7674321320872868,
  'door': 0.8589943455499784,
  'chair': 0.8287681031496973,
  'table': 0.7354849627412889,
  'bookcase': 0.777587422656907,
  'sofa': 0.7144244734782349,
  'board': 0.8014138921354913,
  'clutter': 0.772579039006443},
 'test_full_vote_miou': 79.4476486858435,
 'test_full_vote_iou': {'ceiling': 93.07619949667978,
  'floor': 96.86585232866952,
  'wall': 86.2082081675892,
  'beam': 78.85173561573912,
  'col

In [11]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.35334033025549594,
 'test_loss_cross_entropy': 0.35334033025549594,
 'test_acc': 91.61749473069604,
 'test_macc': 88.41575356281203,
 'test_miou': 78.96436362091487,
 'test_miou_per_class': {0: '94.64',
  1: '97.78',
  2: '86.76',
  3: '77.19',
  4: '54.70',
  5: '72.54',
  6: '81.59',
  7: '82.81',
  8: '73.39',
  9: '75.52',
  10: '71.07',
  11: '83.01',
  12: '75.54'},
 'test_iou': {'ceiling': 0.9464472063348865,
  'floor': 0.977800284413269,
  'wall': 0.8675652219185576,
  'beam': 0.7719067532371585,
  'column': 0.5469994333199425,
  'window': 0.7254085280023362,
  'door': 0.8159029228287055,
  'chair': 0.8280728258818695,
  'table': 0.733893665460212,
  'bookcase': 0.7552016883506514,
  'sofa': 0.7106924119607407,
  'board': 0.8300561742574043,
  'clutter': 0.7554201547531989},
 'test_full_vote_miou': 79.2433080054473,
 'test_full_vote_iou': {'ceiling': 92.92828375557086,
  'floor': 96.8247282211201,
  'wall': 86.23281860559582,
  'beam': 78.9003926772211,
  'c

# Fold 1 - 2cm - XYZRGB*

In [11]:
# 5 votes, 3D aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5220705169747131,
 'test_loss_cross_entropy': 0.5220705169747131,
 'test_acc': 88.17792979412887,
 'test_macc': 82.64148308130707,
 'test_miou': 73.35372445137729,
 'test_miou_per_class': {0: '95.62',
  1: '95.81',
  2: '78.58',
  3: '56.39',
  4: '60.14',
  5: '75.67',
  6: '83.49',
  7: '84.18',
  8: '70.85',
  9: '61.31',
  10: '70.32',
  11: '50.65',
  12: '70.58'},
 'test_iou': {'ceiling': 0.956175917648277,
  'floor': 0.9581325476877459,
  'wall': 0.7857712244933154,
  'beam': 0.5639040312710069,
  'column': 0.6014268919859974,
  'window': 0.7567261590542469,
  'door': 0.8348935752567826,
  'chair': 0.8418057453780545,
  'table': 0.7085305702747743,
  'bookcase': 0.6131074230643703,
  'sofa': 0.7031898334390162,
  'board': 0.5065398050525475,
  'clutter': 0.7057804540729125},
 'test_full_vote_miou': 77.36455944636708,
 'test_full_vote_iou': {'ceiling': 94.22196094761803,
  'floor': 95.52355789525231,
  'wall': 80.83168078993992,
  'beam': 58.05695243885272,
  

# Fold 3

In [54]:
# 10 votes, 3D and 2D aug - half credit - even softer aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3527810864924611,
 'test_loss_cross_entropy': 0.3527810864924611,
 'test_acc': 91.55272704363291,
 'test_macc': 88.71389105935535,
 'test_miou': 79.02833688712079,
 'test_miou_per_class': {0: '94.78',
  1: '97.81',
  2: '86.62',
  3: '75.43',
  4: '55.78',
  5: '71.07',
  6: '81.19',
  7: '83.64',
  8: '74.28',
  9: '73.90',
  10: '73.31',
  11: '84.74',
  12: '74.82'},
 'test_iou': {'ceiling': 0.9477658763190917,
  'floor': 0.9781490240609106,
  'wall': 0.8661768026469115,
  'beam': 0.7542658777352738,
  'column': 0.5577884227329337,
  'window': 0.7106597569880491,
  'door': 0.8119242301016815,
  'chair': 0.8363871535276969,
  'table': 0.7428338305549844,
  'bookcase': 0.7390217871896908,
  'sofa': 0.7330867102333067,
  'board': 0.8473917411559279,
  'clutter': 0.7482325820792443},
 'test_full_vote_miou': 79.12080033147522,
 'test_full_vote_iou': {'ceiling': 92.94930107774319,
  'floor': 96.67482908155745,
  'wall': 85.70187747158654,
  'beam': 77.81115834140073,
 

In [49]:
# 5 votes, 3D and 2D aug - 1/2 credit - more 2D aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3527761208933668,
 'test_loss_cross_entropy': 0.3527761208933668,
 'test_acc': 91.53714926684844,
 'test_macc': 88.65500149676289,
 'test_miou': 78.98911199314071,
 'test_miou_per_class': {0: '94.78',
  1: '97.82',
  2: '86.57',
  3: '75.51',
  4: '55.48',
  5: '71.02',
  6: '81.19',
  7: '83.54',
  8: '74.29',
  9: '73.93',
  10: '73.25',
  11: '84.67',
  12: '74.80'},
 'test_iou': {'ceiling': 0.9477956402903616,
  'floor': 0.9782420603217851,
  'wall': 0.8657194829975473,
  'beam': 0.7550895694272189,
  'column': 0.5548472583903993,
  'window': 0.7102155950591968,
  'door': 0.8118729015628172,
  'chair': 0.8353779052218652,
  'table': 0.742929199777688,
  'bookcase': 0.7392802861098451,
  'sofa': 0.7325037071582057,
  'board': 0.8467395084060412,
  'clutter': 0.7479714443853221},
 'test_full_vote_miou': 79.0324417435195,
 'test_full_vote_iou': {'ceiling': 92.94256408604775,
  'floor': 96.65426653814741,
  'wall': 85.65203998048764,
  'beam': 77.75108010516247,
  '

In [45]:
# 5 votes, 3D and 2D aug - 1/4 credit - barely no 2D aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.35260536010776244,
 'test_loss_cross_entropy': 0.35260536010776244,
 'test_acc': 91.53767016855745,
 'test_macc': 88.64603385350551,
 'test_miou': 78.99699183434147,
 'test_miou_per_class': {0: '94.78',
  1: '97.81',
  2: '86.56',
  3: '75.61',
  4: '55.15',
  5: '71.12',
  6: '81.17',
  7: '83.66',
  8: '74.35',
  9: '73.92',
  10: '73.34',
  11: '84.69',
  12: '74.81'},
 'test_iou': {'ceiling': 0.9477931142893868,
  'floor': 0.9780982404598296,
  'wall': 0.8656150307422625,
  'beam': 0.7560540648798831,
  'column': 0.5514994297941195,
  'window': 0.7111512008458736,
  'door': 0.8116869210294118,
  'chair': 0.8365848765463273,
  'table': 0.7435208584840565,
  'bookcase': 0.7392002805215934,
  'sofa': 0.7334123759414053,
  'board': 0.8468675958324545,
  'clutter': 0.7481249490977856},
 'test_full_vote_miou': 79.05467705156146,
 'test_full_vote_iou': {'ceiling': 92.90218307205502,
  'floor': 96.68680375473369,
  'wall': 85.676266286476,
  'beam': 77.56692360308003,
 

In [41]:
# 5 votes, 3D and 2D aug - half credit - even softer aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3528917676829673,
 'test_loss_cross_entropy': 0.3528917676829673,
 'test_acc': 91.55262849466094,
 'test_macc': 88.68651169198398,
 'test_miou': 79.04611448742223,
 'test_miou_per_class': {0: '94.83',
  1: '97.81',
  2: '86.63',
  3: '76.05',
  4: '55.49',
  5: '71.04',
  6: '81.23',
  7: '83.57',
  8: '74.18',
  9: '73.80',
  10: '73.38',
  11: '84.79',
  12: '74.80'},
 'test_iou': {'ceiling': 0.9482822082138987,
  'floor': 0.9781312901873117,
  'wall': 0.8662631013289761,
  'beam': 0.7604701651204817,
  'column': 0.5549103934753452,
  'window': 0.7104196129717186,
  'door': 0.8122948674558809,
  'chair': 0.8356692519245842,
  'table': 0.7418334597402801,
  'bookcase': 0.7379511493483233,
  'sofa': 0.7338498692968078,
  'board': 0.8479175145332696,
  'clutter': 0.748001999768012},
 'test_full_vote_miou': 79.12692401395431,
 'test_full_vote_iou': {'ceiling': 92.98719117894832,
  'floor': 96.65324350580113,
  'wall': 85.69583557822911,
  'beam': 78.61144860750868,
  

In [35]:
# 5 votes, 3D and 2D aug - half credit - softer aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3519713202273689,
 'test_loss_cross_entropy': 0.3519713202273689,
 'test_acc': 91.56072358878752,
 'test_macc': 88.7109543530574,
 'test_miou': 79.05741337408594,
 'test_miou_per_class': {0: '94.79',
  1: '97.81',
  2: '86.61',
  3: '75.67',
  4: '55.73',
  5: '71.09',
  6: '81.19',
  7: '83.64',
  8: '74.30',
  9: '74.00',
  10: '73.36',
  11: '84.72',
  12: '74.85'},
 'test_iou': {'ceiling': 0.9479189379944415,
  'floor': 0.97813166711946,
  'wall': 0.8660613014729774,
  'beam': 0.7566578059688308,
  'column': 0.5573414459327725,
  'window': 0.7108569856015761,
  'door': 0.8118565277274421,
  'chair': 0.8363690388441877,
  'table': 0.7430198064073663,
  'bookcase': 0.7399953382647456,
  'sofa': 0.7336082042131298,
  'board': 0.8471710445507675,
  'clutter': 0.7484756345334745},
 'test_full_vote_miou': 79.11205367732842,
 'test_full_vote_iou': {'ceiling': 92.94015611054498,
  'floor': 96.66010206728374,
  'wall': 85.70025544947694,
  'beam': 78.0634133310389,
  'co

In [30]:
# 5 votes, 3D and 2D aug - more credit
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3533290109270581,
 'test_loss_cross_entropy': 0.3533290109270581,
 'test_acc': 91.53890906991944,
 'test_macc': 88.69832872371107,
 'test_miou': 79.02852248117121,
 'test_miou_per_class': {0: '94.80',
  1: '97.82',
  2: '86.59',
  3: '75.67',
  4: '55.81',
  5: '71.08',
  6: '81.20',
  7: '83.58',
  8: '74.23',
  9: '73.87',
  10: '73.35',
  11: '84.69',
  12: '74.68'},
 'test_iou': {'ceiling': 0.9479972633202223,
  'floor': 0.9782199640622133,
  'wall': 0.8659159637046069,
  'beam': 0.7566683811239208,
  'column': 0.5581150412758354,
  'window': 0.7107606147577891,
  'door': 0.8120178683662471,
  'chair': 0.8358024704221905,
  'table': 0.7423216848150815,
  'bookcase': 0.7386873534842519,
  'sofa': 0.7335342858395857,
  'board': 0.846856536854947,
  'clutter': 0.746810494525366},
 'test_full_vote_miou': 79.06131212617544,
 'test_full_vote_iou': {'ceiling': 92.9485683980052,
  'floor': 96.67786186650886,
  'wall': 85.70546576827061,
  'beam': 77.81373549366438,
  'c

In [25]:
# 5 votes, 3D and 2D aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3523088596277421,
 'test_loss_cross_entropy': 0.3523088596277421,
 'test_acc': 91.56169500008271,
 'test_macc': 88.7022273249284,
 'test_miou': 79.03073497187705,
 'test_miou_per_class': {0: '94.80',
  1: '97.81',
  2: '86.63',
  3: '75.52',
  4: '55.70',
  5: '71.03',
  6: '81.23',
  7: '83.62',
  8: '74.36',
  9: '73.97',
  10: '73.32',
  11: '84.58',
  12: '74.83'},
 'test_iou': {'ceiling': 0.9479565671461002,
  'floor': 0.9781458658893344,
  'wall': 0.8663127955143941,
  'beam': 0.7551519352481114,
  'column': 0.5570163692589527,
  'window': 0.7103321959429039,
  'door': 0.8123095188826294,
  'chair': 0.8362144530429593,
  'table': 0.7435712730955633,
  'bookcase': 0.7396688675946947,
  'sofa': 0.7331851606182699,
  'board': 0.8458112499743384,
  'clutter': 0.7483192941357647},
 'test_full_vote_miou': 79.09364658957348,
 'test_full_vote_iou': {'ceiling': 92.94280381437953,
  'floor': 96.67307386242396,
  'wall': 85.7086217944507,
  'beam': 77.81256242751911,
  '

In [19]:
# 5 votes, no aug
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.3521801759089744,
 'test_loss_cross_entropy': 0.3521801759089744,
 'test_acc': 91.56051945163128,
 'test_macc': 88.70549025923425,
 'test_miou': 79.05506191068821,
 'test_miou_per_class': {0: '94.79',
  1: '97.82',
  2: '86.62',
  3: '75.48',
  4: '55.90',
  5: '71.09',
  6: '81.19',
  7: '83.61',
  8: '74.28',
  9: '73.98',
  10: '73.31',
  11: '84.83',
  12: '74.81'},
 'test_iou': {'ceiling': 0.9478798869741125,
  'floor': 0.9782495791983941,
  'wall': 0.866194353401721,
  'beam': 0.7548463763700046,
  'column': 0.5590094324535396,
  'window': 0.7108850035359222,
  'door': 0.8118970922462786,
  'chair': 0.8361459999351163,
  'table': 0.742840383699517,
  'bookcase': 0.7397595181014872,
  'sofa': 0.7330763238992631,
  'board': 0.8483226970848722,
  'clutter': 0.7480514014892384},
 'test_vote_miou': 78.735769461432,
 'test_vote_iou': {'ceiling': 91.87428498775748,
  'floor': 95.93093299348196,
  'wall': 84.47802297801847,
  'beam': 76.69279373965662,
  'column': 58.

In [12]:
# trainer._tracker._compute_full_miou()

In [115]:
torch.where(trainer._tracker._test_area.prediction_count > 0)[0].shape[0]

3426294

In [116]:
torch.where(trainer._tracker._test_area.prediction_count > 0)[0].shape[0] / torch.sort(trainer._dataset.test_dataset[0]._datas[0].origin_id).values.shape[0]

0.9999988325592228

In [13]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5434954519626538,
 'test_loss_cross_entropy': 0.5434954519626538,
 'test_acc': 89.34545678736748,
 'test_macc': 72.98235657071672,
 'test_miou': 66.74547850650198,
 'test_miou_per_class': {0: '91.45',
  1: '97.83',
  2: '84.53',
  3: '0.16',
  4: '23.02',
  5: '69.52',
  6: '72.17',
  7: '86.42',
  8: '79.22',
  9: '77.19',
  10: '47.09',
  11: '80.24',
  12: '58.85'},
 'test_iou': {'ceiling': 0.914472306349562,
  'floor': 0.9783106322988362,
  'wall': 0.8453092764997312,
  'beam': 0.0016135625320991955,
  'column': 0.2302064261667585,
  'window': 0.6952422980180125,
  'door': 0.7216831587472861,
  'chair': 0.8641682205448308,
  'table': 0.7921826703968298,
  'bookcase': 0.7718992524432137,
  'sofa': 0.47091768365224684,
  'board': 0.8024107631248552,
  'clutter': 0.5884959550709958},
 'test_full_vote_miou': 66.77704077726602,
 'test_full_vote_iou': {'ceiling': 88.02675251777195,
  'floor': 96.8831018985242,
  'wall': 84.34337034480532,
  'beam': 0.08780188544960096

In [120]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5375950046358123,
 'test_loss_cross_entropy': 0.5375950046358123,
 'test_acc': 89.59099034249877,
 'test_macc': 73.67727153829946,
 'test_miou': 67.48003781030523,
 'test_miou_per_class': {0: '91.55',
  1: '97.90',
  2: '84.56',
  3: '0.04',
  4: '26.73',
  5: '70.85',
  6: '74.79',
  7: '86.70',
  8: '79.83',
  9: '77.44',
  10: '46.98',
  11: '80.89',
  12: '58.99'},
 'test_iou': {'ceiling': 0.915466833857533,
  'floor': 0.9790189737368131,
  'wall': 0.8456038040019893,
  'beam': 0.00042497034968162685,
  'column': 0.2672596656824558,
  'window': 0.7085223612249869,
  'door': 0.7478634101375291,
  'chair': 0.8669852044327894,
  'table': 0.79826439224766,
  'bookcase': 0.7743756806214163,
  'sofa': 0.4697907626799455,
  'board': 0.8089270233610772,
  'clutter': 0.5899018330058045},
 'test_full_vote_miou': 66.64390940373005,
 'test_full_vote_iou': {'ceiling': 87.48702480947411,
  'floor': 96.81578642741313,
  'wall': 84.00539715479997,
  'beam': 0.02341722677611923,

In [93]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5345454134076595,
 'test_acc': 89.71949777070816,
 'test_macc': 74.0860563615468,
 'test_miou': 67.82514231377857,
 'test_miou_per_class': {0: '92.69',
  1: '97.82',
  2: '84.30',
  3: '0.06',
  4: '27.97',
  5: '72.45',
  6: '72.93',
  7: '86.91',
  8: '79.43',
  9: '78.60',
  10: '46.37',
  11: '81.59',
  12: '60.62'},
 'test_vote_miou': 65.72423089183542,
 'test_vote_iou_per_class': {0: '86.29',
  1: '95.90',
  2: '83.05',
  3: '0.10',
  4: '27.09',
  5: '64.64',
  6: '72.39',
  7: '86.02',
  8: '76.82',
  9: '76.30',
  10: '47.90',
  11: '78.60',
  12: '59.33'}}

In [87]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5346095589654787,
 'test_acc': 89.71722095577722,
 'test_macc': 74.0916494826371,
 'test_miou': 67.81836308540296,
 'test_miou_per_class': {0: '92.69',
  1: '97.82',
  2: '84.30',
  3: '0.06',
  4: '28.01',
  5: '72.42',
  6: '72.88',
  7: '86.91',
  8: '79.46',
  9: '78.58',
  10: '46.27',
  11: '81.61',
  12: '60.62'},
 'test_vote_miou': 65.70015265055065,
 'test_vote_iou_per_class': {0: '86.25',
  1: '95.92',
  2: '83.03',
  3: '0.11',
  4: '27.18',
  5: '64.77',
  6: '72.35',
  7: '85.88',
  8: '76.73',
  9: '76.21',
  10: '47.88',
  11: '78.56',
  12: '59.23'}}

In [84]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5346826882837787,
 'test_acc': 89.71763612624306,
 'test_macc': 74.08052939533707,
 'test_miou': 67.8131151702431,
 'test_miou_per_class': {0: '92.68',
  1: '97.81',
  2: '84.30',
  3: '0.06',
  4: '27.94',
  5: '72.43',
  6: '72.91',
  7: '86.89',
  8: '79.43',
  9: '78.61',
  10: '46.25',
  11: '81.62',
  12: '60.62'},
 'test_full_vote_miou': 66.52683527243948,
 'test_full_vote_iou_per_class': {0: '87.36',
  1: '96.71',
  2: '84.04',
  3: '0.10',
  4: '28.23',
  5: '68.46',
  6: '71.69',
  7: '85.89',
  8: '78.51',
  9: '75.90',
  10: '48.86',
  11: '80.27',
  12: '58.83'},
 'test_vote_miou': 65.72050203821799,
 'test_vote_iou_per_class': {0: '86.31',
  1: '95.89',
  2: '83.06',
  3: '0.11',
  4: '27.14',
  5: '64.59',
  6: '72.32',
  7: '86.06',
  8: '76.82',
  9: '76.31',
  10: '47.77',
  11: '78.61',
  12: '59.36'}}

In [77]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5348342829901324,
 'test_acc': 89.71561097129747,
 'test_macc': 74.07859005590113,
 'test_miou': 67.81756718593577,
 'test_miou_per_class': {0: '92.67',
  1: '97.82',
  2: '84.30',
  3: '0.05',
  4: '27.93',
  5: '72.46',
  6: '72.93',
  7: '86.89',
  8: '79.43',
  9: '78.60',
  10: '46.33',
  11: '81.61',
  12: '60.60'},
 'test_full_vote_miou': 66.51553635379987,
 'test_full_vote_iou_per_class': {0: '87.34',
  1: '96.71',
  2: '84.02',
  3: '0.08',
  4: '28.17',
  5: '68.45',
  6: '71.73',
  7: '85.84',
  8: '78.51',
  9: '75.88',
  10: '48.96',
  11: '80.26',
  12: '58.78'},
 'test_vote_miou': 65.71611592024273,
 'test_vote_iou_per_class': {0: '86.28',
  1: '95.90',
  2: '83.04',
  3: '0.08',
  4: '27.06',
  5: '64.60',
  6: '72.37',
  7: '86.02',
  8: '76.83',
  9: '76.30',
  10: '47.92',
  11: '78.59',
  12: '59.32'}}

In [63]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.7711841577788195,
 'test_acc': 85.96227693697763,
 'test_macc': 68.48506462081653,
 'test_miou': 56.481555034279666,
 'test_miou_per_class': {0: '5.58',
  1: '97.64',
  2: '74.47',
  3: '0.00',
  4: '26.20',
  5: '81.87',
  6: '77.29',
  7: '85.97',
  8: '78.95',
  9: '64.79',
  10: '16.65',
  11: '75.79',
  12: '49.05'},
 'test_vote_miou': 54.50633262525753,
 'test_vote_iou_per_class': {0: '1.45',
  1: '94.26',
  2: '74.99',
  3: '0.00',
  4: '27.12',
  5: '81.60',
  6: '78.41',
  7: '83.35',
  8: '71.36',
  9: '64.60',
  10: '14.40',
  11: '71.00',
  12: '46.05'}}

In [58]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.7795100504159929,
 'test_acc': 85.86616218658234,
 'test_macc': 68.68053049988289,
 'test_miou': 56.653280833336005,
 'test_miou_per_class': {0: '6.49',
  1: '97.64',
  2: '73.81',
  3: '0.00',
  4: '25.55',
  5: '82.04',
  6: '77.72',
  7: '85.81',
  8: '78.56',
  9: '64.68',
  10: '17.12',
  11: '78.19',
  12: '48.88'},
 'test_iou_per_class': {'ceiling': 0.06492463311544738,
  'floor': 0.9763536083742955,
  'wall': 0.7381445436394688,
  'beam': 1e-08,
  'column': 0.2554908212952687,
  'window': 0.8204162844481365,
  'door': 0.7771664474138461,
  'chair': 0.8581112769470045,
  'table': 0.7855755458906766,
  'bookcase': 0.6467820622143373,
  'sofa': 0.1712139285445409,
  'board': 0.7819367402579196,
  'clutter': 0.4888106061927395},
 'test_vote_miou': 54.58132131571959,
 'test_vote_iou_per_class': {0: '1.54',
  1: '94.28',
  2: '74.06',
  3: '0.00',
  4: '25.66',
  5: '81.82',
  6: '77.77',
  7: '83.25',
  8: '71.11',
  9: '63.75',
  10: '16.21',
  11: '74.46',
  12

In [43]:
trainer._tracker.get_metrics(True)['test_vote_miou']

54.4874535415312

In [38]:
trainer._tracker.get_metrics(True)['test_vote_miou']

54.98434225685279

In [35]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.7729578473716977,
 'test_acc': 86.01149762819065,
 'test_macc': 68.77769129863522,
 'test_miou': 56.75051956445984,
 'test_miou_per_class': {0: '6.50',
  1: '97.63',
  2: '74.43',
  3: '0.00',
  4: '26.70',
  5: '81.89',
  6: '76.94',
  7: '86.04',
  8: '79.01',
  9: '64.93',
  10: '17.41',
  11: '77.01',
  12: '49.27'},
 'test_iou_per_class': {'ceiling': 0.06497585541060186,
  'floor': 0.9762933174786015,
  'wall': 0.7442718833541156,
  'beam': 1e-08,
  'column': 0.26702636847887407,
  'window': 0.8189266522721443,
  'door': 0.7694081321061437,
  'chair': 0.8603631616795138,
  'table': 0.7900740201738135,
  'bookcase': 0.6492616555753193,
  'sofa': 0.1741042445276164,
  'board': 0.770124351614665,
  'clutter': 0.49273789070837126},
 'test_vote_miou': 54.67553370756352,
 'test_vote_iou_per_class': {0: '1.72',
  1: '94.29',
  2: '74.93',
  3: '0.00',
  4: '27.46',
  5: '81.64',
  6: '77.82',
  7: '83.46',
  8: '71.35',
  9: '64.86',
  10: '14.57',
  11: '72.68',
  12

In [28]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.7796327560146651,
 'test_acc': 85.86740456290943,
 'test_macc': 68.51424123276543,
 'test_miou': 56.49326324598633,
 'test_miou_per_class': {0: '6.38',
  1: '97.63',
  2: '74.43',
  3: '0.00',
  4: '26.36',
  5: '81.95',
  6: '76.56',
  7: '85.85',
  8: '78.83',
  9: '64.38',
  10: '17.75',
  11: '75.34',
  12: '48.95'},
 'test_iou_per_class': {'ceiling': 0.0637840841237654,
  'floor': 0.9763090961862256,
  'wall': 0.7442872684529495,
  'beam': 1e-08,
  'column': 0.2636060323985369,
  'window': 0.8195465679520297,
  'door': 0.7655539094171367,
  'chair': 0.8584893521035004,
  'table': 0.7883127640601619,
  'bookcase': 0.643776916797009,
  'sofa': 0.1775320634562258,
  'board': 0.7534166716346129,
  'clutter': 0.4895094853960687},
 'test_vote_miou': 54.424085545946575,
 'test_vote_iou_per_class': {0: '1.68',
  1: '94.26',
  2: '74.89',
  3: '0.00',
  4: '27.47',
  5: '81.69',
  6: '76.94',
  7: '83.36',
  8: '71.08',
  9: '64.34',
  10: '15.51',
  11: '70.66',
  12: 

In [23]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.7693572307129701,
 'test_acc': 86.06494239891575,
 'test_macc': 68.9100566275572,
 'test_miou': 56.829915964639696,
 'test_miou_per_class': {0: '5.59',
  1: '97.64',
  2: '74.52',
  3: '0.00',
  4: '27.42',
  5: '81.90',
  6: '76.68',
  7: '85.98',
  8: '79.21',
  9: '65.38',
  10: '18.95',
  11: '76.28',
  12: '49.23'},
 'test_iou_per_class': {'ceiling': 0.05593661518075785,
  'floor': 0.976427025013466,
  'wall': 0.7451621007037746,
  'beam': 1e-08,
  'column': 0.2742451895187373,
  'window': 0.8190323636072838,
  'door': 0.7668459050412881,
  'chair': 0.8597656908114887,
  'table': 0.7920553381494594,
  'bookcase': 0.6537818880631856,
  'sofa': 0.1895380991547169,
  'board': 0.7628096129472074,
  'clutter': 0.4922892372117956},
 'test_full_vote_miou': 12.64563421308831,
 'test_full_vote_iou_per_class': {0: '0.72',
  1: '22.17',
  2: '26.88',
  3: '0.00',
  4: '5.78',
  5: '21.49',
  6: '12.07',
  7: '20.58',
  8: '17.38',
  9: '10.94',
  10: '3.11',
  11: '15.29'

In [16]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.7699772521853447,
 'test_acc': 86.05760108425571,
 'test_macc': 68.62170718228533,
 'test_miou': 56.65584448508215,
 'test_miou_per_class': {0: '6.64',
  1: '97.63',
  2: '74.44',
  3: '0.00',
  4: '26.06',
  5: '81.92',
  6: '75.52',
  7: '85.88',
  8: '78.91',
  9: '65.47',
  10: '17.37',
  11: '76.76',
  12: '49.94'},
 'test_iou_ceiling': 0.06636111003972427,
 'test_iou_floor': 0.9762573123073262,
 'test_iou_wall': 0.7443743920925954,
 'test_iou_beam': 1e-08,
 'test_iou_column': 0.2606155318553699,
 'test_iou_window': 0.8191713644295758,
 'test_iou_door': 0.7551890916034606,
 'test_iou_chair': 0.8587555326841981,
 'test_iou_table': 0.7891153464453161,
 'test_iou_bookcase': 0.6547066898577806,
 'test_iou_sofa': 0.17370988403459223,
 'test_iou_board': 0.767628312459297,
 'test_iou_clutter': 0.4993752052514449,
 'test_iou_per_class': {'ceiling': 0.06636111003972427,
  'floor': 0.9762573123073262,
  'wall': 0.7443743920925954,
  'beam': 1e-08,
  'column': 0.260615531

In [13]:
trainer._tracker.get_metrics(True)

{'test_loss_seg': 0.5516111166172086,
 'test_acc': 89.20331614509182,
 'test_macc': 73.11371869835459,
 'test_miou': 66.70007987440918,
 'test_miou_per_class': {0: '92.45',
  1: '97.74',
  2: '83.78',
  3: '0.28',
  4: '24.50',
  5: '71.83',
  6: '67.21',
  7: '86.63',
  8: '78.40',
  9: '77.81',
  10: '46.03',
  11: '80.61',
  12: '59.83'},
 'test_iou_ceiling': 0.9245136199869638,
 'test_iou_floor': 0.9774060218755443,
 'test_iou_wall': 0.8378253314710443,
 'test_iou_beam': 0.002779881843196054,
 'test_iou_column': 0.245009733888497,
 'test_iou_window': 0.718307932941152,
 'test_iou_door': 0.6721132387940401,
 'test_iou_chair': 0.8662854223148914,
 'test_iou_table': 0.7840026738704805,
 'test_iou_bookcase': 0.7780842932351871,
 'test_iou_sofa': 0.46030531913671724,
 'test_iou_board': 0.8061114314045031,
 'test_iou_clutter': 0.5982654829109781,
 'test_iou_per_class': {'ceiling': 0.9245136199869638,
  'floor': 0.9774060218755443,
  'wall': 0.8378253314710443,
  'beam': 0.002779881843196

In [14]:
trainer._tracker.tracked_points

tensor([21798872, 33427063, 29332455,  6303935, 50570005, 71653126,  1606158,
         3613952, 29781089, 28150688, 31826932, 54248365, 31739728, 45256609,
        43518224, 54570038, 55221606, 54579557,  9555843, 54580133, 27309346,
        48981815, 57950985,  6881579, 51460524, 27725190, 24051681, 58389067,
         8528712, 31870660, 35741446, 42253908, 74002003, 34736290,  1539696,
        17629478, 70184933, 47133626, 33804447, 78020475, 51639541, 75635339,
        61900714, 16151101, 69425569, 33048988, 62701500, 32790396, 62757665,
         2085101,  8018284,  8008382, 53760888, 18980330, 53741965, 20814174,
        52677337,  2901075, 57512102, 26885952, 76627927, 34442292, 33877672,
        34470039,  3413249])

In [19]:
trainer._tracker._test_area.votes

tensor([[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        ...,
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [-165.1426, -172.1137,  -72.2697,  ..., -161.6724, -134.5697,
         -132.3147]])

In [55]:
has_prediction = trainer._tracker._test_area.prediction_count > 0

In [56]:
trainer._tracker._test_area.votes[has_prediction].max(dim=1).values

tensor([-0.0806, -0.1203, -0.0934,  ..., -0.2493, -0.9849, -1.0538])

In [57]:
trainer._tracker._test_area.prediction_count[has_prediction].float().mean()

tensor(1.7130)

In [36]:
trainer._tracker._test_area

Batch(_votes=[0], pos=[78649818, 3], prediction_count=[78649818], rgb=[78649818, 3], votes=[78649818, 13], y=[78649818])

In [37]:
trainer._tracker._test_area.y[has_prediction]

tensor([0, 0, 0,  ..., 5, 5, 5])

In [44]:
a = torch.arange(3)
b = torch.LongTensor([1, 0, 1, 1, 1])
print(a)
a[b] += 1
print(a)

tensor([0, 1, 2])
tensor([1, 2, 2])


In [9]:
# 3d-only train aug, 3 runs
trainer._tracker.get_metrics(verbose=True)

{'test_loss_seg': 0.5410348689952293,
 'test_acc': 89.37068044068191,
 'test_macc': 73.42683686131298,
 'test_miou': 67.1288373763577,
 'test_miou_per_class': {0: '92.40',
  1: '97.71',
  2: '83.87',
  3: '0.13',
  4: '27.50',
  5: '72.48',
  6: '68.87',
  7: '86.66',
  8: '78.80',
  9: '78.31',
  10: '45.19',
  11: '80.57',
  12: '60.19'},
 'test_iou_per_class': {'ceiling': 0.924015897279107,
  'floor': 0.977122361139335,
  'wall': 0.83867233475217,
  'beam': 0.001252350823969891,
  'column': 0.2749623583000838,
  'window': 0.7248315130134239,
  'door': 0.6886631795011481,
  'chair': 0.8665891781027597,
  'table': 0.7880008707466383,
  'bookcase': 0.7831452598054893,
  'sofa': 0.45188094081135316,
  'board': 0.8057357079783476,
  'clutter': 0.6018769066726736},
 'test_full_vote_miou': 66.59925256007342,
 'test_vote_miou': 65.8572664590206}

In [18]:
# 2d and 3d train aug, 3 runs
trainer._tracker.get_metrics(verbose=True)

{'test_loss_seg': 0.5404121148843498,
 'test_acc': 89.50866059750476,
 'test_macc': 73.59207708304231,
 'test_miou': 67.27609980508969,
 'test_miou_per_class': {0: '92.61',
  1: '97.80',
  2: '83.96',
  3: '0.08',
  4: '27.49',
  5: '72.44',
  6: '70.19',
  7: '86.60',
  8: '79.05',
  9: '78.39',
  10: '44.96',
  11: '80.90',
  12: '60.11'},
 'test_iou_per_class': {'ceiling': 0.9260538873625491,
  'floor': 0.9780452902152748,
  'wall': 0.8395811750825389,
  'beam': 0.0008161474640181739,
  'column': 0.27489026614735895,
  'window': 0.7244173371388533,
  'door': 0.701917350789978,
  'chair': 0.8660155645109545,
  'table': 0.7905473854901168,
  'bookcase': 0.7838704299397383,
  'sofa': 0.44961489533098664,
  'board': 0.8090098352081287,
  'clutter': 0.6011134099811641},
 'test_full_vote_miou': 66.4098997046715,
 'test_vote_miou': 65.61836098409874}

In [10]:
# no aug, 5 runs
trainer._tracker.get_metrics(verbose=True)

{'test_loss_seg': 0.5408402153787512,
 'test_acc': 89.50441800294236,
 'test_macc': 73.59545438316832,
 'test_miou': 67.27720273454156,
 'test_miou_per_class': {0: '92.58',
  1: '97.81',
  2: '83.94',
  3: '0.08',
  4: '27.57',
  5: '72.45',
  6: '69.89',
  7: '86.60',
  8: '79.09',
  9: '78.33',
  10: '45.32',
  11: '80.66',
  12: '60.29'},
 'test_iou_per_class': {'ceiling': 0.9258343955677176,
  'floor': 0.9780577311500906,
  'wall': 0.839358624152682,
  'beam': 0.0007660827965585151,
  'column': 0.2756606379398916,
  'window': 0.7245281349191418,
  'door': 0.6988789939915359,
  'chair': 0.8659848598513187,
  'table': 0.7909276117883951,
  'bookcase': 0.7833419557268848,
  'sofa': 0.45319550945739834,
  'board': 0.8066082129571315,
  'clutter': 0.6028936051916562},
 'test_full_vote_miou': 66.42841975738816,
 'test_vote_miou': 65.65106117369578}

In [13]:
{i: "{:.2f}".format(100 * v) for i, v in enumerate(trainer._tracker.full_confusion_matrix.get_intersection_union_per_class()[0])}

{0: '87.35',
 1: '96.72',
 2: '84.00',
 3: '0.07',
 4: '27.96',
 5: '68.48',
 6: '70.97',
 7: '85.83',
 8: '78.41',
 9: '75.93',
 10: '48.83',
 11: '80.06',
 12: '58.95'}

In [7]:
# no aug, basic res, 5 runs
trainer._tracker.get_metrics(verbose=True)

{'test_loss_seg': 0.5412074728576183,
 'test_acc': 89.50644041748885,
 'test_macc': 73.587772263621,
 'test_miou': 67.29529259575797,
 'test_miou_per_class': {0: '92.58',
  1: '97.80',
  2: '83.97',
  3: '0.06',
  4: '27.57',
  5: '72.43',
  6: '69.70',
  7: '86.66',
  8: '79.16',
  9: '78.39',
  10: '45.57',
  11: '80.74',
  12: '60.20'},
 'test_iou_per_class': {'ceiling': 0.9258076660422176,
  'floor': 0.9780228338128133,
  'wall': 0.8396710805088385,
  'beam': 0.0005957507807477139,
  'column': 0.27570810501349496,
  'window': 0.7243341686440086,
  'door': 0.6970477207935556,
  'chair': 0.8666226944076589,
  'table': 0.7916377044422881,
  'bookcase': 0.7838542925434459,
  'sofa': 0.4556885954791953,
  'board': 0.8074328002440686,
  'clutter': 0.6019646247362049}}

In [16]:
trainer._tracker._confusion_matrix.get_average_intersection_union()

0.655690671652356

In [17]:
trainer._tracker._vote_miou

63.477841096249676

In [19]:
trainer._tracker.get_metrics(verbose=True)

{'test_loss_seg': 0.6443870441913605,
 'test_acc': 87.76017878375214,
 'test_macc': 78.09775459845794,
 'test_miou': 65.5690671652356,
 'test_miou_per_class': {0: '91.51',
  1: '97.47',
  2: '82.13',
  3: '0.00',
  4: '33.56',
  5: '75.56',
  6: '68.70',
  7: '84.71',
  8: '80.33',
  9: '73.13',
  10: '44.35',
  11: '67.40',
  12: '53.55'},
 'test_iou_per_class': {'ceiling': 0.9151453575260218,
  'floor': 0.9747499295300937,
  'wall': 0.8212637117565389,
  'beam': 1e-08,
  'column': 0.3355951342466767,
  'window': 0.7555801801509413,
  'door': 0.6869869797490971,
  'chair': 0.8470596286627782,
  'table': 0.8033196831874237,
  'bookcase': 0.7312838087102287,
  'sofa': 0.4435011391197315,
  'board': 0.6739585007560484,
  'clutter': 0.5355346680850476},
 'test_full_vote_miou': 24.981851752918445,
 'test_vote_miou': 63.477841096249676}

In [23]:
trainer._tracker._confusion_matrix.confusion_matrix.sum()

4398610

In [25]:
trainer._tracker._test_area

Batch(pos=[78649818, 3], prediction_count=[78649818], rgb=[78649818, 3], votes=[78649818, 13], y=[78649818])

In [36]:
trainer._dataset.test_data.

Batch(pos=[78649818, 3], rgb=[78649818, 3], y=[78649818])

### Pick-100 test set inference 
| Augmentations | Runs | Full Res | mIoU |
| --- | :-: | :-: | --- |
|  | 1 | | 64.16 - 63.71 |
|  |  |  |  |
|  | 1 | X | 64.16 |
|  | 3 | X | 63.78 |
|  | 5 | X | 63.79 |
|  | 10 | X | 63.64 |
|  |  |  |  |
| credit /2 | 1 | | 63.85 - 63.58 - 63.94 |
|  |  |  |  |
| credit x2 | 1 | | 63.76 - 64.12 - 63.37 - 64.29 |
| credit x2 | 3 | | 64.10 |
| credit x2 | 5 | | 63.92 |
|  |  |  |  |
| credit x4 | 1 | | 63.36 - 63.66 |
|  |  |  |  |
| 2D | 1 | | 63.35 |
| 2D | 3 | | 63.61 |
| 2D | 5 | | 63.64 |
| 2D | 10 | | 63.91 |
|  |  |  |  |
| 2D credit x2 | 1 | | 63.93 |
| 2D credit x2 | 3 | | 63.97 |
| 2D credit x2 | 5 | | 63.93 |
| 2D credit x2 | 10 | | 63.92 |
|  |  |  |  |
| 2D less aug | 1 | | 63.73 |
| 2D less aug | 3 | | 63.85 |
| 2D less aug | 5 | | 63.91 |
|  |  |  |  |
| 2D more aug | 1 | | 64.03 |
| 2D more aug | 3 | | 63.75 |
| 2D more aug | 5 | | 63.78 |
|  |  |  |  |
| 3D | 1 | | 63.90 |
| 3D | 3 | | 63.97 |
| 3D | 5 | | 63.95 |
| 3D | 10 | | 63.95 |
|  |  |  |  |
| 3D | 1 | X | ... |
| 3D | 3 | X | ... |
| 3D | 5 | X | ... |
| 3D | 10 | X | ... |
|  |  |  |  |
| 2D-3D | 1 | | 63.46 |
| 2D-3D | 3 | | 63.52 |
| 2D-3D | 5 | | 63.74 |
|  |  |  |  |
| 2D-3D | 1 | X | 63.79 |
| 2D-3D | 3 | X | 63.57 |
| 2D-3D | 5 | X | 63.82 |
| 2D-3D | 10 | X | 63.93 |
|  |  |  |  |
| 2D-3D use_coverage=F | 1 | X | 64.10 |
| 2D-3D use_coverage=F | 3 | X | 64.18 |
| 2D-3D use_coverage=F | 5 | X | 64.19 |
| 2D-3D use_coverage=F | 10 | X | 63.95 |

### Full test set inference 
| Augmentations | Runs | Full Res | mIoU |
| --- | :-: | :-: | --- |
|  | 1 | | 65.35 |
|  | 1 | X | 65.24 |### Full test set inference 
| Augmentations | Runs | Full Res | mIoU |
| --- | :-: | :-: | --- |
|  | 1 | | 65.35 |
|  | 1 | X | 65.24 |

Baseline for ResUNet34-L4-L0-128 on **subsampled** with **no test-time augmentation**: 65.2 mIoU

In [12]:
trainer._tracker.get_metrics(verbose=True)

{'test_loss_seg': 0.5730561992951806,
 'test_acc': 88.10689588334989,
 'test_macc': 72.34868877204623,
 'test_miou': 64.62987632054187,
 'test_miou_per_class': {0: '89.34',
  1: '97.55',
  2: '83.45',
  3: '0.86',
  4: '27.14',
  5: '67.68',
  6: '63.00',
  7: '84.49',
  8: '77.12',
  9: '77.29',
  10: '35.22',
  11: '80.43',
  12: '56.62'},
 'test_iou_per_class': {'ceiling': 0.8934271580623707,
  'floor': 0.9755455915401167,
  'wall': 0.8345440965341843,
  'beam': 0.008577480072643449,
  'column': 0.27135430753183987,
  'window': 0.6767822486359989,
  'door': 0.6299961243108358,
  'chair': 0.8449028076383593,
  'table': 0.7711804741239968,
  'bookcase': 0.7728727069848975,
  'sofa': 0.3522145729588146,
  'board': 0.8043144622628593,
  'clutter': 0.5661718910135253}}

In [11]:
trainer._tracker.get_metrics(verbose=True)

{'test_loss_seg': 0.5723548031389898,
 'test_acc': 88.13282837348112,
 'test_macc': 72.61964483560868,
 'test_miou': 64.66669811700491,
 'test_miou_per_class': {0: '89.26',
  1: '97.60',
  2: '83.51',
  3: '1.13',
  4: '27.75',
  5: '67.42',
  6: '63.37',
  7: '84.30',
  8: '77.21',
  9: '77.50',
  10: '35.31',
  11: '79.60',
  12: '56.72'},
 'test_iou_per_class': {'ceiling': 0.8926215214058315,
  'floor': 0.9759997636240926,
  'wall': 0.8350906267563485,
  'beam': 0.011254983397804447,
  'column': 0.2774807927353927,
  'window': 0.6741813953507453,
  'door': 0.6337058446348253,
  'chair': 0.8430078008616986,
  'table': 0.7720666335623955,
  'bookcase': 0.7749868587032627,
  'sofa': 0.35314519834336944,
  'board': 0.7959514996659558,
  'clutter': 0.5671778361689144},
 'test_full_vote_miou': 64.82117443883027,
 'test_vote_miou': 64.00841317943247}

In [13]:
trainer._tracker._vote_miou, trainer._tracker._full_vote_miou

(64.00841317943247, 64.82117443883027)